In [8]:
#for autoEncDec
import pyperclip

py_cmd = ['py','python','python3'][2]
image_path = './data/images/'
img_count = -1
epochs = 300
batch_size = 8
loss = 'msle'
gpu_num = 3
cmd =   f"""
{py_cmd} autoEncDec.py --image_path {image_path} 
--img_count {img_count} --epochs {epochs} 
--batch_size {batch_size} --loss {loss} 
--gpu_num {gpu_num}
"""
cmd = cmd.replace('\n','')

pyperclip.copy(cmd)
cmd

'python3 autoEncDec.py --image_path ./data/images/ --img_count -1 --epochs 300 --batch_size 8 --loss msle --gpu_num 3'

In [10]:
#for test_model
import pyperclip

py_cmd = ['py','python','python3'][2]

seq_len = 24
n_seq = 6
hidden_dim = 6
gamma = 1

noise_dim = 32
dim = 128
batch_size = 8

log_step = 100
lr = 5e-4
gpu_num = 3

image_path = './data/images/'
img_count = -1

cmd =   f"""
{py_cmd} autoEncDec.py --seq_len {seq_len} --n_seq {n_seq} 
--hidden_dim {hidden_dim} --gamma {gamma} 
--noise_dim {noise_dim} --dim {dim} --batch_size {batch_size} 
--log_step {log_step} --lr {lr} --gpu_num {gpu_num} 
--image_path {image_path} --img_count {img_count}
"""
cmd = cmd.replace('\n','')

pyperclip.copy(cmd)
cmd

'python3 autoEncDec.py --seq_len 24 --n_seq 6 --hidden_dim 6 --gamma 1 --noise_dim 32 --dim 128 --batch_size 8 --log_step 100 --lr 0.0005 --gpu_num 3 --image_path ./data/images/ --img_count -1'

In [1]:
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

from ydata_synthetic.synthesizers import ModelParameters
from ydata_synthetic.preprocessing.timeseries import processed_stock
from ydata_synthetic.synthesizers.timeseries import TimeGAN

from results_save import result_compile_mnist

c:\Users\Tardy\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
fld = f"./timeGAN_models/mnist/2022-08-19-21-46/timeGAN_model.pk1"
synth = TimeGAN.load(fld)



In [3]:
synth_data = synth.sample(40)



Synthetic data generation: 100%|██████████| 6/6 [00:01<00:00,  3.65it/s]


In [4]:
synth_data.shape

(48, 24, 784)

In [17]:
n,s,_ = synth_data.shape
from PIL import Image

In [24]:
bckg = Image.new(mode="RGB",size=(410,160),color=(255,255,255))
# result_compile_mnist(synth_data,'2022-08-19-21-46',n,s)
img = Image.fromarray(synth_data[0][0*8+0].reshape((28,28)))
bckg.paste(img.resize((40,40)),(10,10))
bckg.save('./temp/0.png')